<a href="https://colab.research.google.com/github/sightes/BIGDATA-UBO2022/blob/main/Pyspark_Streaming_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark 
import os 
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.3.1-bin-hadoop3'
import findspark 
findspark.init()          

In [37]:
spark.stop()

In [38]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
.master('local')\
.appName('Ejemplo1')\
.config('spark.ui.port',4050).getOrCreate()

In [39]:
from pyspark.sql.types import * 
schema=StructType([StructField('ID',IntegerType(),True)])

In [40]:
!mkdir streaming 

mkdir: cannot create directory ‘streaming’: File exists


In [41]:
people_df=spark.readStream\
.format('csv').schema(schema)\
.option('header',True).load('streaming')

In [ ]:
people_df.isStreaming

True

In [42]:
n=0

In [43]:
%%python --bg
import time
import pandas as pd 
import numpy as np
def aleatorio():
  return int(99*np.random.normal(0, 0.1)+1)
while True:
  pd.DataFrame(np.asarray(
      [aleatorio() for i in range(0,100)]).reshape(-1,1),columns=['ID'])\
  .to_csv('/content/streaming/data'+ str(n)+'.csv')
  n=n+1
  time.sleep(10)

In [44]:
result_df=people_df.select('*')
query=(result_df.writeStream.format('json').queryName('selectTable')\
       .option('checkpointLocation','checkpoint')\
       .option('path','results')\
       .start()\
       .awaitTermination())


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored